In [80]:
import numpy as np

In [81]:
class LinearModel:
    
    def __init__(self, A = np.empty([0,0]), b = np.empty([0,0]), c = np.empty([0,0]), minmax = "MAX"):
        self.A = A
        self.b = b
        self.c = c
        self.x = [float(0)] * len(c)
        self.minmax = minmax
        self.printIter = True
        self.optimalValue = None
        self.transform = False
        
    def addA(self, A):
        self.A = A
        
    def addB(self, b):
        self.b = b
        
    def addC(self, c):
        self.c = c
        self.transform = False
    
    def setObj(self, minmax):
        if(minmax == "MIN" or minmax == "MAX"):
            self.minmax = minmax
        else:
            print("Invalid objective.")
        self.transform = False
            
    def setPrintIter(self, printIter):
        self.printIter = printIter
            
    def printSoln(self):
        print("Coefficients: ")
        print(self.x)
        print("Optimal value: ")
        print(self.optimalValue)
        
    def printTableau(self, tableau):
        
        print("ind \t\t", end = "")
        for j in range(0, len(c)):
            print("x_" + str(j), end = "\t")
        for j in range(0, (len(tableau[0]) - len(c) - 2)):
            print("s_" + str(j), end = "\t")
        
        print()
        for j in range(0, len(tableau)):
            for i in range(0, len(tableau[0])):
                if(not np.isnan(tableau[j, i])):
                    if(i == 0):
                        print(int(tableau[j, i]), end = "\t")
                    else:
                        print(round(tableau[j, i], 2), end = "\t")
                else:
                    print(end = "\t")
            print()
            
    def getTableau(self):
        # construct starting tableau
        
        if(self.minmax == "MIN" and self.transform == False):
            self.c[0:len(c)] = -1 * self.c[0:len(c)]
            self.transform = True
        
        t1 = np.array([None, 0])
        numVar = len(self.c)
        numSlack = len(self.A)
        
        t1 = np.hstack(([None], [0], self.c, [0] * numSlack))
        
        basis = np.array([0] * numSlack)
        
        for i in range(0, len(basis)):
            basis[i] = numVar + i
        
        A = self.A
        
        if(not ((numSlack + numVar) == len(self.A[0]))):
            B = np.identity(numSlack)
            A = np.hstack((self.A, B))
            
        t2 = np.hstack((np.transpose([basis]), np.transpose([self.b]), A))
        
        tableau = np.vstack((t1, t2))
        
        tableau = np.array(tableau, dtype ='float')
        
        return tableau
            
    def optimize(self):
        
        if(self.minmax == "MIN" and self.transform == False):
            for i in range(len(self.c)):
                self.c[i] = -1 * self.c[i]
                transform = True
        
        tableau = self.getTableau()
         
        if(self.printIter == True):
            print("Starting Tableau:")
            self.printTableau(tableau)
        
        # assume initial basis is not optimal
        optimal = False

        # keep track of iterations for display
        iter = 1

        while(True):
            
            if(self.printIter == True):
                print("----------------------------------")
                print("Iteration :", iter)
                self.printTableau(tableau)
                
            if(self.minmax == "MAX"):
                for profit in tableau[0, 2:]:
                    if profit > 0:
                        optimal = False
                        break
                    optimal = True
            else:
                for cost in tableau[0, 2:]:
                    if cost < 0:
                        optimal = False
                        break
                    optimal = True

            # if all directions result in decreased profit or increased cost
            if optimal == True: 
                 break
            
            # nth variable enters basis, account for tableau indexing
            if (self.minmax == "MAX"):
                n = tableau[0, 2:].tolist().index(np.amax(tableau[0, 2:])) + 2
            else:
                n = tableau[0, 2:].tolist().index(np.amin(tableau[0, 2:])) + 2

            # minimum ratio test, rth variable leaves basis 
            minimum = 99999
            r = -1

            for i in range(1, len(tableau)): 
                if(tableau[i, n] > 0):
                    val = tableau[i, 1]/tableau[i, n]
                    if val<minimum: 
                        minimum = val 
                        r = i
                            
            pivot = tableau[r, n] 
            
            print("Pivot Column:", n)
            print("Pivot Row:", r)
            print("Pivot Element: ", pivot)

            # perform row operations 
            # divide the pivot row with the pivot element 
            tableau[r, 1:] = tableau[r, 1:] / pivot 
            
            

            # pivot other rows
            for i in range(0, len(tableau)): 
                if i != r:
                    mult = tableau[i, n] / tableau[r, n]
                    tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 


            # new basic variable 
            tableau[r, 0] = n - 2
            
            iter += 1
            
        
        if(self.printIter == True):
            print("----------------------------------")
            print("Final Tableau reached in", iter, "iterations")
            self.printTableau(tableau)
        else:
            print("Solved")
            
        self.x = np.array([0] * len(c), dtype = float)
        # save coefficients
        for key in range(1, (len(tableau))):
            if(tableau[key, 0] < len(c)):
                self.x[int(tableau[key, 0])] = tableau[key, 1]
        
        self.optimalValue = -1 * tableau[0,1]

In [82]:
model1 = LinearModel()

A = np.array([[1, 1], 
              [0, 1], 
              [1, 2]])
b = np.array([6, 3, 9])
c = np.array([2, 5])

model1.addA(A)
model1.addB(b)
model1.addC(c)

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[1 1]
 [0 1]
 [1 2]] 

b =
 [6 3 9] 

c =
 [2 5] 


Starting Tableau:
ind 		x_0	x_1	s_0	s_1	s_2	
	0.0	2.0	5.0	0.0	0.0	0.0	
2	6.0	1.0	1.0	1.0	0.0	0.0	
3	3.0	0.0	1.0	0.0	1.0	0.0	
4	9.0	1.0	2.0	0.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	s_0	s_1	s_2	
	0.0	2.0	5.0	0.0	0.0	0.0	
2	6.0	1.0	1.0	1.0	0.0	0.0	
3	3.0	0.0	1.0	0.0	1.0	0.0	
4	9.0	1.0	2.0	0.0	0.0	1.0	
Pivot Column: 3
Pivot Row: 2
Pivot Element:  1.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	s_0	s_1	s_2	
	-15.0	2.0	0.0	0.0	-5.0	0.0	
2	3.0	1.0	0.0	1.0	-1.0	0.0	
1	3.0	0.0	1.0	0.0	1.0	0.0	
4	3.0	1.0	0.0	0.0	-2.0	1.0	
Pivot Column: 2
Pivot Row: 1
Pivot Element:  1.0
----------------------------------
Iteration : 3
ind 		x_0	x_1	s_0	s_1	s_2	
	-21.0	0.0	0.0	-2.0	-3.0	0.0	
0	3.0	1.0	0.0	1.0	-1.0	0.0	
1	3.0	0.0	1.0	0.0	1.0	0.0	
4	0.0	0.0	0.0	-1.0	-1.0	1.0	
----------------------------------
Final Tableau reached in 3 iterations
ind 		x_0	x_1	s_0	s_1	s_2	
	-21.0	0.0	0.0	-2.0	-3.0	0.0	
0

In [83]:
A = np.array([[4, 7, 9], 
              [1, 4, 5], 
              [9, 0, 3]])
b = np.array([4, 7, 2])
c = np.array([3, 3, 3])

model1.addA(A)
model1.addB(b)
model1.addC(c)

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[4 7 9]
 [1 4 5]
 [9 0 3]] 

b =
 [4 7 2] 

c =
 [3 3 3] 


Starting Tableau:
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	0.0	3.0	3.0	3.0	0.0	0.0	0.0	
3	4.0	4.0	7.0	9.0	1.0	0.0	0.0	
4	7.0	1.0	4.0	5.0	0.0	1.0	0.0	
5	2.0	9.0	0.0	3.0	0.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	0.0	3.0	3.0	3.0	0.0	0.0	0.0	
3	4.0	4.0	7.0	9.0	1.0	0.0	0.0	
4	7.0	1.0	4.0	5.0	0.0	1.0	0.0	
5	2.0	9.0	0.0	3.0	0.0	0.0	1.0	
Pivot Column: 2
Pivot Row: 3
Pivot Element:  9.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	-0.67	0.0	3.0	2.0	0.0	0.0	-0.33	
3	3.11	0.0	7.0	7.67	1.0	0.0	-0.44	
4	6.78	0.0	4.0	4.67	0.0	1.0	-0.11	
0	0.22	1.0	0.0	0.33	0.0	0.0	0.11	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  7.0
----------------------------------
Iteration : 3
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	-2.0	0.0	0.0	-1.29	-0.43	0.0	-0.14	
1	0.44	0.0	1.0	1.1	0.14	0.0	-0.06	
4	5.0	0.0	0.0	0.29	-0.57	1.0	0.14	
0	0.22	1.0	0.0	0.33	0.0	0.0	0.11	
----------------------

In [84]:
A = np.array([[8, 1, 0, 1], 
              [3, 4, 1, 0]])
b = np.array([12, 6])
c = np.array([1, 2])

model1.addA(A)
model1.addB(b)
model1.addC(c)

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[8 1 0 1]
 [3 4 1 0]] 

b =
 [12  6] 

c =
 [1 2] 


Starting Tableau:
ind 		x_0	x_1	s_0	s_1	
	0.0	1.0	2.0	0.0	0.0	
2	12.0	8.0	1.0	0.0	1.0	
3	6.0	3.0	4.0	1.0	0.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	s_0	s_1	
	0.0	1.0	2.0	0.0	0.0	
2	12.0	8.0	1.0	0.0	1.0	
3	6.0	3.0	4.0	1.0	0.0	
Pivot Column: 3
Pivot Row: 2
Pivot Element:  4.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	s_0	s_1	
	-3.0	-0.5	0.0	-0.5	0.0	
2	10.5	7.25	0.0	-0.25	1.0	
1	1.5	0.75	1.0	0.25	0.0	
----------------------------------
Final Tableau reached in 2 iterations
ind 		x_0	x_1	s_0	s_1	
	-3.0	-0.5	0.0	-0.5	0.0	
2	10.5	7.25	0.0	-0.25	1.0	
1	1.5	0.75	1.0	0.25	0.0	


Coefficients: 
[0.  1.5]
Optimal value: 
3.0


In [85]:
A = np.array([[1, 2, 2], 
              [2, 1, 2], 
              [2, 2, 1]])
b = np.array([20, 20, 20])
c = np.array([-10, -12, -12])

model1.addA(A)
model1.addB(b)
model1.addC(c)
model1.setObj("MIN")

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")
model1.optimize()
print("\n")
model1.printSoln()

A =
 [[1 2 2]
 [2 1 2]
 [2 2 1]] 

b =
 [20 20 20] 

c =
 [-10 -12 -12] 


Starting Tableau:
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	0.0	-10.0	-12.0	-12.0	0.0	0.0	0.0	
3	20.0	1.0	2.0	2.0	1.0	0.0	0.0	
4	20.0	2.0	1.0	2.0	0.0	1.0	0.0	
5	20.0	2.0	2.0	1.0	0.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	0.0	-10.0	-12.0	-12.0	0.0	0.0	0.0	
3	20.0	1.0	2.0	2.0	1.0	0.0	0.0	
4	20.0	2.0	1.0	2.0	0.0	1.0	0.0	
5	20.0	2.0	2.0	1.0	0.0	0.0	1.0	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  2.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	120.0	-4.0	0.0	0.0	6.0	0.0	0.0	
1	10.0	0.5	1.0	1.0	0.5	0.0	0.0	
4	10.0	1.5	0.0	1.0	-0.5	1.0	0.0	
5	0.0	1.0	0.0	-1.0	-1.0	0.0	1.0	
Pivot Column: 2
Pivot Row: 3
Pivot Element:  1.0
----------------------------------
Iteration : 3
ind 		x_0	x_1	x_2	s_0	s_1	s_2	
	120.0	0.0	0.0	-4.0	2.0	0.0	4.0	
1	10.0	0.0	1.0	1.5	1.0	0.0	-0.5	
4	10.0	0.0	0.0	2.5	1.0	1.0	-1.5	
0	0.0	1.0	0.0	-1.0	-1.0	0.0	1.0	
Pivot Colum